In [ ]:
# import modules

import ee
from osgeo import gdal
from osgeo import gdalconst
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy import ndimage
from scipy.stats import linregress
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

In [ ]:
# initialize the ee api through credentials

# ee.Authenticate()
ee.Initialize()

In [ ]:
# load study area

fc = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
roi = fc.filter(ee.Filter.eq('ADM1_NAME', 'Zuid-holland'))

In [ ]:
# transfer images from ee to numpy array (for intersecting dates)
sentinel_dates = ['2020-03-25']
sar_arrs = []

for dates in sentinel_dates:
    sentinel = ee.ImageCollection('COPERNICUS/S1_GRD')
    asc = sentinel.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).filter(ee.Filter.eq('instrumentMode', 'IW'))
    platform = asc.filter(ee.Filter.eq('platform_number', 'A'))
    coll_param = platform.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).select(['VV', 'VH'])

    sentinel_roi = coll_param.filterBounds(roi.geometry())

    bcoff = sentinel_roi.filterDate(dates, '2021-01-01').first()

    bcoff_new = bcoff.reduceResolution(reducer=ee.Reducer.median(), maxPixels=1e4).reproject(crs='EPSG:4326', scale=1000)

    sar_arr = bcoff_new.sampleRectangle(region=roi.geometry(), defaultValue=-9999)

    sar_arr_VV = sar_arr.get('VV')
    sar_arr_VH = sar_arr.get('VH')

    npsar_arr_VV = np.array(sar_arr_VV.getInfo())
    sar_arrs.append(npsar_arr_VV)
    npsar_arr_VH = np.array(sar_arr_VH.getInfo())
    sar_arrs.append(npsar_arr_VH)

    print(npsar_arr_VV.shape)
    print(npsar_arr_VH.shape)

In [ ]:
# prepare arrays for plotting

vv_arr = sar_arrs[0]
print('The shape of the vv_backscatter array is:', vv_arr.shape)
print()

vh_arr = sar_arrs[1]
print('The shape of the vv_backscatter array is:', vv_arr.shape)
print()

In [ ]:
# get data for extent correction

extent_data = gdal.Open('data/modisval_2905.tif')
geoTransform = extent_data.GetGeoTransform()
ulx = geoTransform[0]
uly = geoTransform[3]
lrx = ulx + geoTransform[1] * extent_data.RasterXSize
lry = uly + geoTransform[5] * extent_data.RasterYSize
print(ulx, uly, lrx, lry)

In [ ]:
# get landsat validation data and cut by extent

lst_full = gdal.Open('data/l8/landsatval_2503_100.tif')
tmp_data = gdal.Translate('/vsimem/in_memory_output.tif', lst_full, projWin=[ulx, uly, lrx, lry],
                          outputType=gdalconst.GDT_Float32, noData=np.nan)
lst_full_arr = tmp_data.ReadAsArray()
lst_full_arr = lst_full_arr*0.00341802+149.0
lst_full_farr = ndimage.median_filter(lst_full_arr, 3)
print(lst_full_farr.shape)

In [ ]:
# save lst 1000 m

l8_1000 = lst_full_farr.reshape(-1, 10, 131, 10)
l8_1000_m = np.median(l8_1000, (-1, -3))
lst_arr = l8_1000_m

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.imshow(lst_arr, cmap='RdBu_r')
plt.colorbar(orientation='horizontal')
plt.title('LST image over Zuid-Holland (1000 m)', y=-0.1)
plt.show()

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.imshow(vv_arr, cmap='Greys_r')
plt.colorbar(orientation='horizontal')
plt.title('VV image over Zuid-Holland (1000 m)', y=-0.1)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 20))

img_1 = ax[0].imshow(lst_arr, cmap='RdBu_r')
fig.colorbar(img_1, ax=ax[0], orientation='horizontal')
ax[0].set_title('LST image over Zuid-Holland (1000 m)', y=-0.5)

img_2 = ax[1].imshow(vv_arr, cmap='Greys_r')
fig.colorbar(img_2, ax=ax[1], orientation='horizontal')
ax[1].set_title('VV image over Zuid-Holland (1000 m)', y=-0.5)

fig.show()

## SAR feature processing

In [ ]:
# get sar data and cut it by modis extent

sar_full = gdal.Open('data/s1/sarval_2503_vv_vh.tif')
tmp_data_sar = gdal.Translate('/vsimem/in_memory_output.tif', sar_full, projWin=[ulx, uly, lrx, lry],
                              outputType=gdalconst.GDT_Float32, noData=np.nan)
vv_full_arr = tmp_data_sar.ReadAsArray()[0]
vv_full_farr = ndimage.median_filter(vv_full_arr, 3)
print(vv_full_farr.shape)

vh_full_arr = tmp_data_sar.ReadAsArray()[1]
vh_full_farr = ndimage.median_filter(vh_full_arr, 3)
print(vh_full_farr.shape)

In [ ]:
# upscale to 100 m res

n_vv_full_arr = vv_full_farr.reshape(-1, 10, 1310, 10)
m_vv_full_arr = np.median(n_vv_full_arr, (-1, -3))
print(m_vv_full_arr.shape)
print(m_vv_full_arr)

# upscale to 100 m res

n_vh_full_arr = vh_full_farr.reshape(-1, 10, 1310, 10)
m_vh_full_arr = np.median(n_vh_full_arr, (-1, -3))
print(m_vh_full_arr.shape)
print(m_vh_full_arr)

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.imshow(m_vh_full_arr, cmap='RdBu_r')
plt.colorbar(orientation='horizontal')
plt.title('VV image over Zuid-Holland (100 m)', y=-0.1)
plt.show()

### Scale and remove nan

In [ ]:
# remove nan
vv_arr[np.isnan(vv_arr)] = np.nanmean(vv_arr)
m_vv_full_arr[np.isnan(m_vv_full_arr)] = np.nanmean(m_vv_full_arr)
vv_full_farr[np.isnan(vv_full_farr)] = np.nanmean(vv_full_farr)

vh_arr[np.isnan(vh_arr)] = np.nanmean(vh_arr)
m_vh_full_arr[np.isnan(m_vh_full_arr)] = np.nanmean(m_vh_full_arr)
vh_full_farr[np.isnan(vh_full_farr)] = np.nanmean(vh_full_farr)

# scale
vv_arr_norm = (vv_arr - np.nanmin(vv_arr))/(np.nanmax(vv_arr) - np.nanmin(vv_arr))
m_vv_full_arr_norm = (m_vv_full_arr - np.nanmin(m_vv_full_arr))/(np.nanmax(m_vv_full_arr) - np.nanmin(m_vv_full_arr))
vv_full_farr_norm = (vv_full_farr - np.nanmin(vv_full_farr))/(np.nanmax(vv_full_farr) - np.nanmin(vv_full_farr))

vh_arr_norm = (vh_arr - np.nanmin(vh_arr))/(np.nanmax(vh_arr) - np.nanmin(vh_arr))
m_vh_full_arr_norm = (m_vh_full_arr - np.nanmin(m_vh_full_arr))/(np.nanmax(m_vh_full_arr) - np.nanmin(m_vh_full_arr))
vh_full_farr_norm = (vh_full_farr - np.nanmin(vh_full_farr))/(np.nanmax(vh_full_farr) - np.nanmin(vh_full_farr))

/for reg,
vv_arr = vv_arr at 1000 m,
m_vv_full_arr = vv_arr at 100 m

/for cnn,
vv_patches = (10, 10) patches from 100 m vv image,
vv_patches_100 = (10, 10) patches from 10 m vv image

In [ ]:
# patches

vv_patches = [m_vv_full_arr_norm[x:x+10, y:y+10].reshape(10, 10, 1) for x in range(0, m_vv_full_arr_norm.shape[0], 10) for y in range(0, m_vv_full_arr_norm.shape[1], 10)]
with tf.device('cpu:0'):
    vv_patches_tensor = tf.convert_to_tensor(vv_patches)
print(vv_patches_tensor.shape)

vh_patches = [m_vh_full_arr_norm[x:x+10, y:y+10].reshape(10, 10, 1) for x in range(0, m_vh_full_arr_norm.shape[0], 10) for y in range(0, m_vh_full_arr_norm.shape[1], 10)]
with tf.device('cpu:0'):
    vh_patches_tensor = tf.convert_to_tensor(vh_patches)
print(vh_patches_tensor.shape)

vv_patches_100 = [vv_full_farr_norm[x:x+10, y:y+10].reshape(10, 10, 1) for x in range(0, vv_full_farr_norm.shape[0], 10) for y in range(0, vv_full_farr_norm.shape[1], 10)]
with tf.device('cpu:0'):
    vv_patches_tensor_100 = tf.convert_to_tensor(vv_patches_100)
print(vv_patches_tensor_100.shape)

vh_patches_100 = [vh_full_farr_norm[x:x+10, y:y+10].reshape(10, 10, 1) for x in range(0, vh_full_farr_norm.shape[0], 10) for y in range(0, vh_full_farr_norm.shape[1], 10)]
with tf.device('cpu:0'):
    vh_patches_tensor_100 = tf.convert_to_tensor(vh_patches_100)
print(vh_patches_tensor_100.shape)

## S2

### uncomment the below code to use patches from sentinel-2 data as input to the model

In [ ]:
# # s2 1000 m product

# s2_arrs = []

# s2_data = gdal.Open('data/s2/s2_2603_1000.tif')

# for i in range(s2_data.RasterCount):
#     s2_arrs.append(s2_data.ReadAsArray()[i])

In [ ]:
# # get s2 data and cut it by extent

# s2_full_arrs = []

# s2_full = gdal.Open('data/s2/s2_2603_10.tif')
# tmp_data_s2 = gdal.Translate('/vsimem/in_memory_output.tif', s2_full, projWin=[ulx, uly, lrx, lry],
#                               outputType=gdalconst.GDT_Float32, noData=np.nan)

# for i in range(tmp_data_s2.RasterCount):
#     arr = ndimage.median_filter(tmp_data_s2.ReadAsArray()[i], 3)
#     s2_full_arrs.append(arr)

In [ ]:
# # upscale to 100 m res

# s2_full_arrs_100 = []

# for arr in s2_full_arrs:
#     n_arr = arr.reshape(-1, 10, 1310, 10)
#     m_n_arr = np.median(n_arr, (-1, -3))
#     s2_full_arrs_100.append(m_n_arr)

In [ ]:
# print(len(s2_arrs), len(s2_full_arrs), len(s2_full_arrs_100))

In [ ]:
# # remove nan and scale
# s2_arrs_norm = []
# for arr in s2_arrs:
#     arr[np.isnan(arr)] = np.nanmean(arr)
#     arr_norm = (arr - np.nanmin(arr))/(np.nanmax(arr) - np.nanmin(arr))
#     s2_arrs_norm.append(arr_norm)

# s2_full_arrs_norm = []
# for arr in s2_full_arrs:
#     arr[np.isnan(arr)] = np.nanmean(arr)
#     arr_norm = (arr - np.nanmin(arr))/(np.nanmax(arr) - np.nanmin(arr))
#     s2_full_arrs_norm.append(arr_norm)

# s2_full_arrs_100_norm = []
# for arr in s2_full_arrs_100:
#     arr[np.isnan(arr)] = np.nanmean(arr)
#     arr_norm = (arr - np.nanmin(arr))/(np.nanmax(arr) - np.nanmin(arr))
#     s2_full_arrs_100_norm.append(arr_norm)

In [ ]:
# print(len(s2_arrs_norm), len(s2_full_arrs_norm), len(s2_full_arrs_100_norm))

In [ ]:
# s2_10_patches = []
# for arr in s2_full_arrs_norm:
#     arr_patch = [arr[x:x+10, y:y+10].reshape(10, 10, 1) for x in range(0, arr.shape[0], 10) for y in range(0, arr.shape[1], 10)]
#     with tf.device('cpu:0'):
#         arr_tensor = tf.convert_to_tensor(arr_patch)
#     s2_10_patches.append(arr_tensor)

# s2_100_patches = []
# for arr in s2_full_arrs_100_norm:
#     arr_patch = [arr[x:x+10, y:y+10].reshape(10, 10, 1) for x in range(0, arr.shape[0], 10) for y in range(0, arr.shape[1], 10)]
#     with tf.device('cpu:0'):
#         arr_tensor = tf.convert_to_tensor(arr_patch)
#     s2_100_patches.append(arr_tensor)

In [ ]:
# with tf.device('cpu:0'):
#     s2_patches_tensor_10 = tf.concat(s2_10_patches, axis=-1)
# print(s2_patches_tensor_10.shape)

# with tf.device('cpu:0'):
#     s2_patches_tensor = tf.concat(s2_100_patches, axis=-1)
# print(s2_patches_tensor.shape)

## LULC

In [ ]:
lulc_data = gdal.Open('data/esa_lulc_100.tif')
tmp_data_lulc = gdal.Translate('/vsimem/in_memory_output.tif', lulc_data, projWin=[ulx, uly, lrx, lry],
                              outputType=gdalconst.GDT_Float32, noData=np.nan)
lulc_arr = tmp_data_lulc.ReadAsArray()
print(lulc_arr.shape)

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.imshow(lulc_arr)
plt.colorbar(orientation='horizontal')
plt.title('LULC image over Zuid-Holland (100 m)', y=-0.1)
plt.show()

In [ ]:
lulc_data_10 = gdal.Open('data/esa_lulc_10.tif')
tmp_data_lulc = gdal.Translate('/vsimem/in_memory_output.tif', lulc_data_10, projWin=[ulx, uly, lrx, lry],
                              outputType=gdalconst.GDT_Float32, noData=np.nan)
lulc_arr_10 = tmp_data_lulc.ReadAsArray()
print(lulc_arr_10.shape)

In [ ]:
# remove nan and normalize
lulc_arr[np.isnan(lulc_arr)] = 80.0
lulc_arr_10[np.isnan(lulc_arr_10)] = 80.0

# normalize
lulc_arr_norm = (lulc_arr - np.nanmin(lulc_arr))/(np.nanmax(lulc_arr) - np.nanmin(lulc_arr))
lulc_arr_10_norm = (lulc_arr_10 - np.nanmin(lulc_arr_10))/(np.nanmax(lulc_arr_10) - np.nanmin(lulc_arr_10))

In [ ]:
lulc_patches = [lulc_arr_norm[x:x+10, y:y+10].reshape(10, 10, 1) for x in range(0, lulc_arr_norm.shape[0], 10) for y in range(0, lulc_arr_norm.shape[1], 10)]
with tf.device('cpu:0'):
    lulc_patches_tensor = tf.convert_to_tensor(lulc_patches)
print(lulc_patches_tensor.shape)

lulc_patches_100 = [lulc_arr_10_norm[x:x+10, y:y+10].reshape(10, 10, 1) for x in range(0, lulc_arr_10_norm.shape[0], 10) for y in range(0, lulc_arr_10_norm.shape[1], 10)]
with tf.device('cpu:0'):
    lulc_patches_tensor_100 = tf.convert_to_tensor(lulc_patches_100)
print(lulc_patches_tensor_100.shape)

## predictors and target

In [ ]:
# stack tensors cnn

predictor_tensor = tf.concat([lulc_patches_tensor, vv_patches_tensor, vh_patches_tensor], axis=-1)
# predictor_tensor = s2_patches_tensor
print(predictor_tensor.shape)

# target
lst_arr_norm = (lst_arr - np.nanmin(lst_arr))/(np.nanmax(lst_arr) - np.nanmin(lst_arr))
target_values = lst_arr_norm.flatten()
target_tensor = tf.convert_to_tensor(target_values)
print(target_tensor.shape)

**Here the shape of the predictor is (num of samples, patch_size_x, patch_size_y, num of predictors). The idea is to map a (10, 10) patch at 100 m resolution predictor image to the corresponding coarse resolution pixel at 1000 m**

**for prediction, (10, 10) patches are collected again but now from 10 m image to correspondingly estimate target values at 100 m**

In [ ]:
# remove nan

# Find the indices of the NaN values in the target array
nan_indices = np.isnan(target_tensor)

# Remove the corresponding rows from the predictor and target arrays
predictor_tensor = predictor_tensor[~nan_indices]
target_tensor = target_tensor[~nan_indices]

print(predictor_tensor.shape)
print(target_tensor.shape)

## Building Model:

In [ ]:
# Define the model architecture

# cnn
input_shape_1 = (10, 10, 3)  # patch size of predictor image
input_1 = Input(shape=input_shape_1, name='input_1')

x = Conv2D(32, (10, 10), activation='relu', padding='same')(input_1)
x = Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
x = Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (5, 5), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(16, (3, 3),  strides=(1, 1), activation='relu', padding='same')(x)
x = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
output_1 = Dense(1)(x)

model = Model(inputs=input_1, outputs=output_1)
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
# # model viz

# tf.keras.utils.plot_model(model, to_file='cnn_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
with tf.device('cpu:0'):
    model.fit(predictor_tensor, target_tensor, epochs=30, batch_size=32)

In [ ]:
# stack tensors for prediction
with tf.device("cpu:0"):
    predictor_tensor_10 = tf.concat([lulc_patches_tensor_100, vv_patches_tensor_100, vh_patches_tensor_100], axis=-1)
    # predictor_tensor_10 = s2_patches_tensor_10

print(predictor_tensor_10.shape)

In [ ]:
with tf.device("cpu:0"):
    m_pred = model.predict(predictor_tensor_10)

In [ ]:
dlst_norm = m_pred.reshape(-1, 1310)

# inverse scaling
dlst = dlst_norm * (np.nanmax(lst_arr) - np.nanmin(lst_arr)) + np.nanmin(lst_arr)

figure(figsize=(14, 12), dpi=150)
plt.imshow(dlst, cmap='RdBu_r')
plt.colorbar(orientation='horizontal')
plt.show()

In [ ]:
print(np.nanmin(dlst), np.nanmax(dlst))
print(np.nanmin(dlst_norm), np.nanmax(dlst_norm))

In [ ]:
# residual correction

fulllst_pred_1000 = dlst.reshape(-1, 10, 131, 10)
fulllst_pred_1000 = np.median(fulllst_pred_1000, (-1, -3))
print(fulllst_pred_1000.shape)

# residuals

res_1000 = lst_arr - fulllst_pred_1000
print(res_1000.shape)

res_100 = res_1000.repeat(10, 0).repeat(10, 1)
print(res_100.shape)
print(res_100)

plt.imshow(res_1000)
plt.colorbar()

In [ ]:
# add residuals to the prediction

dlst_res = dlst + res_100
print(dlst_res.shape)

In [ ]:
# qualitative validation

min_min = np.nanmin(dlst_res)
max_max = np.nanmax(dlst_res)

figure(figsize=(14, 12), dpi=300)

plt.subplot(1, 2, 1)
plt.imshow(dlst_res, vmin=min_min, vmax=max_max, cmap='RdBu_r')
plt.title('Downscaled LST map')
plt.colorbar(orientation='horizontal')

plt.subplot(1, 2, 2)
plt.imshow(lst_full_farr, vmin=min_min, vmax=max_max, cmap='RdBu_r')
plt.title('Original LST map')
plt.colorbar(orientation='horizontal')
plt.show()

In [ ]:
# quantitative

val_df = pd.DataFrame(dlst.flatten().T, columns=['dlst'])
val_df['dlst_res'] = dlst_res.flatten().T
val_df['lst'] = lst_full_farr.flatten().T
val_df['lulc'] = lulc_arr.flatten().T

val_df

In [ ]:
# remove lulc = 0 class

val_df.loc[val_df['lst'].isnull(), :] = np.nan
val_df

In [ ]:
# hist plot
hist_df = val_df[['lst', 'dlst']]

figure(figsize=(14, 12), dpi=300)
hist_plot = hist_df.plot.hist(bins=200, legend=True, alpha=0.5)
fig = hist_plot.get_figure()
fig.show()

In [ ]:
# error for each pixel

error_full_arr = np.sqrt(np.square(dlst_res - lst_full_farr))

figure(figsize=(14, 12), dpi=150)
plt.title('Error at each pixel')
plt.imshow(error_full_arr, cmap='RdYlGn_r')
plt.colorbar(orientation='horizontal')
plt.show()

In [ ]:
corr_data_df = val_df[['lst', 'dlst', 'dlst_res', 'lulc']]
corr_data_df = corr_data_df.dropna()

print('The correlation between Observed LST and Downscaled LST at 100m is:',
      corr_data_df['lst'].corr(corr_data_df['dlst_res']))

In [ ]:
rmse_before = ((corr_data_df.dlst - corr_data_df.lst) ** 2).mean() ** .5
print('RMSE before residual correction:', rmse_before)
rmse_after = ((corr_data_df.dlst_res - corr_data_df.lst) ** 2).mean() ** .5
print('RMSE after residual correction:', rmse_after)

In [ ]:
slope, intercept, r_value, p_value, std_err = linregress(corr_data_df['lst'], corr_data_df['dlst_res'])
r_squared = r_value ** 2

print('R^2:', r_squared)

In [ ]:
corr_data_df_full = pd.DataFrame(data=None, columns=corr_data_df.columns, index=val_df.index)
corr_data_df_full

In [ ]:
corr_data_df_full = corr_data_df_full.combine_first(corr_data_df)
corr_data_df_full

In [ ]:
def save_as_tif(img_name, src_arr, mask_img):
    mask_data = gdal.Open(mask_img)
    driverTiff = gdal.GetDriverByName('GTiff')
    clfds = driverTiff.Create(img_name,
                              mask_data.RasterXSize, mask_data.RasterYSize,
                              1, gdal.GDT_Float32)
    clfds.SetGeoTransform(mask_data.GetGeoTransform())
    clfds.SetProjection(mask_data.GetProjection())
    clfds.GetRasterBand(1).SetNoDataValue(-9999.0)
    clfds.GetRasterBand(1).WriteArray(src_arr)
    clfds = None

In [ ]:
dlst_n = corr_data_df_full.dlst.values.reshape(-1, 1310)
dlst_n_res = corr_data_df_full.dlst_res.values.reshape(-1, 1310)

In [ ]:
# save_as_tif('dlst_res.tif', dlst_n_res, 'data/saving_mask_100.tif')